In [ ]:
!pip install pyldavis

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
import nltk
from transformers import BertTokenizer, BertModel
import torch
import os
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
from sklearn.cluster import KMeans
import numpy as np
import ssl
from gensim.models.ldamodel import LdaModel

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def get_bert_embeddings(text):
    inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True, padding=True)
    with torch.no_grad():  # Disable gradient calculations for efficiency
        outputs = model(**inputs)
    # Take mean of all token embeddings to get a single vector for the text
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()


# Example
text = "You can have the money to pay for but if you're foreclosed the banks will much rather prefer to take the house anyway and give it to someone else. My parents just officially lost their house after years of fighting it. Haven't missed a payment during foreclosure. House foreclosed during the housing bubble burst when Bank of America illegally foreclosed on a bunch of people early. They missed one payment in their entire lifetime but it was at that time. They were able to go bankrupt, pay all the legal fees for that and trying to keep the house plus pay the house and minimum life needs. When the bank wants your house there's little you can do. "
embeddings = get_bert_embeddings(text)

directory = './files/filtered_json_files_2'

# filename = 'truist_combined.json'
for filename in os.listdir(directory):
    if filename.endswith('.json'):
        output_data = []
        texts = []
        with open(os.path.join(directory, filename), 'r', encoding='utf-8') as file:
            for line in file:
                # Find the 'body' field in the JSON line
                body_header = line.find('"body": "')
                if body_header != -1:
                    # Extract the content of the 'body' field
                    body_text_start = body_header + 9
                    body_text_end = line.find('"', body_text_start)
                    tweet = line[body_text_start:body_text_end]
                    texts.append(tweet)


try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('stopwords')
stop_words = stopwords.words('english')

# Function to preprocess text
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in stop_words and len(token) > 3:
            result.append(token)
    return result

# Preprocess the texts
processed_texts = [preprocess(text) for text in texts]

# Create a dictionary and corpus needed for LDA
dictionary = corpora.Dictionary(processed_texts)
corpus = [dictionary.doc2bow(text) for text in processed_texts]

num_topics = 2

# Build LDA model
lda_model = LdaModel(corpus=corpus,
                     id2word=dictionary,
                     num_topics=num_topics,
                     random_state=100,
                     update_every=1,
                     chunksize=100,
                     passes=10,
                     alpha='auto')

# View the topics in LDA model
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

# Prepare the visualization
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, dictionary, sort_topics=False)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Topic: 0 
Words: 0.008*"financial" + 0.008*"bank" + 0.007*"corp" + 0.006*"risk" + 0.005*"banks" + 0.005*"market" + 0.005*"blockchain" + 0.004*"technology" + 0.004*"group" + 0.004*"said"
Topic: 1 
Words: 0.010*"bank" + 0.007*"market" + 0.005*"fund" + 0.005*"management" + 0.004*"banks" + 0.004*"mellon" + 0.004*"financial" + 0.004*"nthe" + 0.003*"would" + 0.003*"https"


In [ ]:
# Display the visualization
pyLDAvis.display(vis)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
num_topics = 3

# Build LDA model
lda_model = LdaModel(corpus=corpus,
                     id2word=dictionary,
                     num_topics=num_topics,
                     random_state=100,
                     update_every=1,
                     chunksize=100,
                     passes=10,
                     alpha='auto')

# View the topics in LDA model
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

# Prepare the visualization
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, dictionary, sort_topics=False)
# Display the visualization
pyLDAvis.display(vis)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Topic: 0 
Words: 0.010*"risk" + 0.008*"blockchain" + 0.007*"financial" + 0.006*"technology" + 0.005*"bank" + 0.005*"energy" + 0.005*"operational" + 0.005*"banks" + 0.005*"investors" + 0.004*"management"
Topic: 1 
Words: 0.012*"bank" + 0.008*"market" + 0.007*"banks" + 0.006*"fund" + 0.005*"cash" + 0.005*"management" + 0.004*"financial" + 0.004*"clients" + 0.004*"mellon" + 0.004*"nthe"
Topic: 2 
Words: 0.009*"corp" + 0.008*"bank" + 0.007*"financial" + 0.006*"market" + 0.006*"group" + 0.005*"corporation" + 0.005*"said" + 0.004*"services" + 0.004*"year" + 0.004*"stocks"


In [ ]:
num_topics = 4

# Build LDA model
lda_model = LdaModel(corpus=corpus,
                     id2word=dictionary,
                     num_topics=num_topics,
                     random_state=100,
                     update_every=1,
                     chunksize=100,
                     passes=10,
                     alpha='auto')

# View the topics in LDA model
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

# Prepare the visualization
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, dictionary, sort_topics=False)
# Display the visualization
pyLDAvis.display(vis)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Topic: 0 
Words: 0.011*"risk" + 0.009*"blockchain" + 0.008*"market" + 0.007*"bank" + 0.007*"technology" + 0.006*"financial" + 0.006*"banks" + 0.006*"operational" + 0.005*"http" + 0.005*"investors"
Topic: 1 
Words: 0.016*"bank" + 0.008*"fund" + 0.007*"banks" + 0.007*"nthe" + 0.006*"management" + 0.005*"market" + 0.005*"said" + 0.005*"would" + 0.005*"luxembourg" + 0.005*"clients"
Topic: 2 
Words: 0.010*"corp" + 0.009*"market" + 0.007*"financial" + 0.006*"bank" + 0.006*"corporation" + 0.005*"group" + 0.005*"year" + 0.005*"services" + 0.005*"stocks" + 0.004*"management"
Topic: 3 
Words: 0.008*"energy" + 0.007*"financial" + 0.005*"bank" + 0.005*"company" + 0.004*"banks" + 0.004*"group" + 0.004*"properties" + 0.004*"realty" + 0.004*"broker" + 0.004*"bancorp"


In [ ]:
num_topics = 5

# Build LDA model
lda_model = LdaModel(corpus=corpus,
                     id2word=dictionary,
                     num_topics=num_topics,
                     random_state=100,
                     update_every=1,
                     chunksize=100,
                     passes=10,
                     alpha='auto')

# View the topics in LDA model
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

# Prepare the visualization
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, dictionary, sort_topics=False)
# Display the visualization
pyLDAvis.display(vis)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Topic: 0 
Words: 0.014*"blockchain" + 0.011*"market" + 0.009*"technology" + 0.008*"year" + 0.007*"stocks" + 0.006*"banks" + 0.006*"bank" + 0.006*"said" + 0.005*"billion" + 0.004*"many"
Topic: 1 
Words: 0.018*"bank" + 0.010*"market" + 0.007*"banks" + 0.006*"said" + 0.006*"nthe" + 0.006*"cash" + 0.006*"cent" + 0.005*"https" + 0.005*"would" + 0.005*"deposits"
Topic: 2 
Words: 0.008*"bank" + 0.007*"corp" + 0.006*"management" + 0.006*"services" + 0.006*"mellon" + 0.006*"account" + 0.006*"fidelity" + 0.005*"corporation" + 0.005*"morgan" + 0.005*"international"
Topic: 3 
Words: 0.013*"financial" + 0.010*"corp" + 0.008*"group" + 0.007*"banks" + 0.007*"bank" + 0.007*"energy" + 0.006*"company" + 0.005*"capital" + 0.005*"corporation" + 0.004*"america"
Topic: 4 
Words: 0.017*"risk" + 0.011*"fund" + 0.011*"management" + 0.007*"investors" + 0.007*"luxembourg" + 0.007*"operational" + 0.007*"asset" + 0.006*"investment" + 0.005*"buffalo" + 0.005*"index"


In [ ]:
num_topics = 6

# Build LDA model
lda_model = LdaModel(corpus=corpus,
                     id2word=dictionary,
                     num_topics=num_topics,
                     random_state=100,
                     update_every=1,
                     chunksize=100,
                     passes=10,
                     alpha='auto')

# View the topics in LDA model
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

# Prepare the visualization
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, dictionary, sort_topics=False)
# Display the visualization
pyLDAvis.display(vis)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Topic: 0 
Words: 0.011*"bank" + 0.008*"banks" + 0.007*"lehman" + 0.005*"fdic" + 0.004*"market" + 0.004*"blockchain" + 0.004*"mellon" + 0.004*"billion" + 0.004*"reserve" + 0.004*"treasury"
Topic: 1 
Words: 0.011*"fund" + 0.008*"market" + 0.008*"management" + 0.007*"bank" + 0.007*"cash" + 0.006*"luxembourg" + 0.006*"clients" + 0.006*"https" + 0.006*"banks" + 0.005*"investment"
Topic: 2 
Words: 0.011*"bank" + 0.011*"financial" + 0.011*"corp" + 0.008*"group" + 0.007*"cent" + 0.006*"services" + 0.006*"franc" + 0.006*"swiss" + 0.006*"corporation" + 0.005*"account"
Topic: 3 
Words: 0.010*"energy" + 0.008*"financial" + 0.005*"company" + 0.005*"bank" + 0.005*"group" + 0.005*"properties" + 0.005*"realty" + 0.004*"bancorp" + 0.004*"broker" + 0.004*"firm"
Topic: 4 
Words: 0.029*"bank" + 0.012*"buffalo" + 0.011*"pittsburgh" + 0.007*"first" + 0.007*"like" + 0.005*"cities" + 0.005*"financial" + 0.005*"national" + 0.004*"mellon" + 0.004*"make"
Topic: 5 
Words: 0.011*"market" + 0.010*"risk" + 0.007*"bl